In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20,5)})

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv('../input/cusersmarildownloadsscarcitycsv/scarcity.csv', delimiter=';')
df.head()

In [ ]:
df.describe()

In [ ]:
df[df['population']==0]

In [ ]:
df = df.drop(df[df['population']==0].index)

In [ ]:
dict = {}
for col in list(df.columns):
    dict[col] = df[col].value_counts().shape[0]

pd.DataFrame(dict,index=["unique count"])

In [ ]:
cont_col = ['population', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 
            'average', 'low', 'moderate', 'significant', 'severe']
comma = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec','average']

for i in comma:
    df[i] = df[i].str.replace(',', '').astype(float)


In [ ]:
plt.figure(figsize=(20, 15))
for i, column in enumerate(cont_col, 1):
    plt.subplot(4, 5, i)
    sns.kdeplot(x=df[column],color='indianred')
    plt.legend()
    plt.xlabel(column)

In [ ]:
df = df.drop('basin_id',axis=1)

In [ ]:
conditions = ['low','moderate','significant','severe']
plt.figure(figsize=(20, 15))
i = 1
for column in conditions:
    plt.subplot(len(conditions), 1, i)
    sns.violinplot(x=df[column],y=df['population'])
    plt.xlabel(column)
    i += 1
plt.show()

In [ ]:
conditions = ['low','moderate','significant','severe']
plt.figure(figsize=(20, 15))
i = 1
for column in conditions:
    plt.subplot(len(conditions), 1, i)
    sns.violinplot(x=df[column],y=df['average'])
    plt.xlabel(column)
    i += 1
plt.show()

In [ ]:
check = 10
plt.figure(figsize=(20,8))
df_ave = df.copy()
df_ave = df_ave.set_index('basin_name')
df_ave['population'].sort_values().tail(check).plot(kind='barh',color='r')
plt.title(f'The {check} basins with largest population')

In [ ]:
df.sort_values(by='population',ascending=False).head(check)

In [ ]:
plt.figure(figsize=(20,8))
df_ave['average'].sort_values().tail(check).plot(kind='barh',color='r')
plt.title(f'The {check} bains with highest average value ')

In [ ]:
df.sort_values(by='average',ascending=False).head(check)

In [ ]:
most_severe = df_ave[df_ave['severe'] == 12].sort_values(by='average',ascending=False).head(check)
plt.figure(figsize=(20,7))
most_severe['average'].sort_values().plot(kind='barh',color='r')
plt.title(f'Top {check} of severe water scarcity countries')

In [ ]:
least_severe = df_ave[df_ave['low'] == 12].sort_values(by='average',ascending=False).head(check)
plt.figure(figsize=(20,7))
least_severe['average'].plot(kind='barh',color='r')
plt.title(f'Top {check} of low water scarcity countries')

In [ ]:
df_long = pd.melt(df, id_vars=['basin_name','population'], value_vars=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
                  var_name='months', value_name='values')
df_long.head()

In [ ]:
sns.kdeplot(x=df_long['values'],hue=df_long['months'])

In [ ]:
df_months = df_long.groupby(by='months').sum()
df_months['values'].sort_values().plot(kind='barh',color='r')
plt.title(f'Top month of severe water scarcity')

In [ ]:
df['population_log'] = np.log10(df['population'])
df['average_log'] = np.log10(df['average'])
x = df[['population_log','average_log']]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

In [ ]:
from sklearn.cluster import KMeans
wcss = []

for i in range(1,8):
    kmeans = KMeans(i)
    kmeans.fit(x)
    wcss_iter = kmeans.inertia_
    wcss.append(wcss_iter)

number_of_cluster = range(1,8)
plt.plot(number_of_cluster,wcss)

In [ ]:

kmeans = KMeans(3)
kmeans.fit(x)
identified_clusters = kmeans.fit_predict(x)
data_with_clusters = df.copy()
data_with_clusters['Cluster'] = identified_clusters
plt.figure(figsize=(10,10))
plt.scatter(data_with_clusters['population_log'],data_with_clusters['average_log'],c=data_with_clusters['Cluster'],cmap='rainbow')
plt.xlabel('population_log')
plt.ylabel('average_log')